In [2]:
import time
import psutil
import threading

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score
)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from dataloader import load_data
import warnings
warnings.simplefilter("ignore")

Data directory: d:\DSS5104\data


In [ ]:
def preprocess_tabular_data(X_df, y, cat_cols, cont_cols, is_train=False, scaler=None, encoder=None):
    X = X_df.copy()

    if cat_cols:
        X[cat_cols] = X[cat_cols].astype(str)

        if is_train:
            encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
            X[cat_cols] = encoder.fit_transform(X[cat_cols])
        else:
            X[cat_cols] = encoder.transform(X[cat_cols])
    else:
        encoder = None

    if cont_cols:
        X[cont_cols] = X[cont_cols].astype("float32")
        if is_train:
            scaler = StandardScaler()
            X[cont_cols] = scaler.fit_transform(X[cont_cols])
        else:
            X[cont_cols] = scaler.transform(X[cont_cols])
    else:
        scaler = None

    return X, np.array(y), scaler, encoder


def prepare_data_np(dataset_name: str):
    dataset_name = dataset_name.lower()

    if dataset_name.startswith("adult"):
        '''
        you can select a seed:777, 888, 999, 
        and change the seed 'build_model(task_type: str, seed = 999)' at the same time
        '''
        data_train, data_test = load_data("adult", seed = 999) 
        X_train, y_train = data_train.drop(columns=['income']), (data_train['income'] == '>50K').astype(int)
        X_val, y_val = data_test.drop(columns=['income']), (data_test['income'] == '>50K').astype(int)
        task_type = "classification"

    elif dataset_name.startswith("california"):
        X_train, X_val, y_train, y_val = load_data("california", seed = 777)
        task_type = "regression"

    elif dataset_name.startswith("higgs"):
        X_train, X_val, y_train, y_val = load_data("higgs")
        y_train, y_val = (y_train == 1).astype(int), (y_val == 1).astype(int)
        task_type = "classification"

    elif dataset_name.startswith("churn"):
        X_train, X_val, y_train, y_val = load_data("churn")
        y_train, y_val = (y_train == 'Yes').astype(int), (y_val == 'Yes').astype(int)
        task_type = "classification"

    elif dataset_name.startswith("creditcard"):
        X_train, X_val, y_train, y_val = load_data("credit")
        y_train, y_val = (y_train == 1).astype(int), (y_val == 1).astype(int)
        task_type = "classification"

    elif dataset_name.startswith("poker"):
        X_train, X_val, y_train, y_val = load_data("poker")
        task_type = "classification"

    elif dataset_name.startswith("bank"):
        X_train, X_val, y_train, y_val = load_data("bank")
        y_train, y_val = (y_train == 'yes').astype(int), (y_val == 'yes').astype(int)
        task_type = "classification"

    elif dataset_name.startswith("wine"):
        X_train, X_val, y_train, y_val = load_data("wine")
        task_type = "regression"

    elif dataset_name.startswith("covtype"):
        X_train, X_val, y_train, y_val = load_data("covtype")
        task_type = "classification"

    else:
        raise ValueError(f"Unsupported dataset: {dataset_name}")

    cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
    cont_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

    X_train_processed, y_train_array, scaler, encoder = preprocess_tabular_data(
        X_train, y_train, cat_cols, cont_cols, is_train=True)

    X_val_processed, y_val_array, _, _ = preprocess_tabular_data(
        X_val, y_val, cat_cols, cont_cols, is_train=False, scaler=scaler, encoder=encoder)

    return (X_train_processed, y_train_array), (X_val_processed, y_val_array), task_type


def build_model(task_type: str, seed = 999):
    if task_type == "classification":
        model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=seed, n_jobs=-1)
    elif task_type == "regression":
        model = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=seed, n_jobs=-1)
    else:
        raise ValueError(f"Unsupported task type: {task_type}")
    
    return model


def evaluate_model(model, X_test, y_test, task_type, train_time, avg_cpu_usage):
    y_test_pred = model.predict(X_test)

    results = {
        "train_time_seconds": round(train_time, 4),
        "cpu_usage": round(avg_cpu_usage, 4)
    }

    print("\nEvaluation Results:")

    if task_type == "classification":
        acc = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, average='macro')
        results.update({
            "test_accuracy": round(acc, 4),
            "test_f1": round(f1, 4),
        })
        
        print(f"Accuracy: {acc:.4f}")
        print(f"F1 Score: {f1:.4f}")

        try:
            y_proba = model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_proba[:, 1])
            results["test_auc"] = round(auc, 4)
            print(f"AUC: {auc:.4f}")
        except Exception as e:
            results["test_auc"] = np.nan
            print("AUC can't be calculated:", e)

    else:  # regression
        rmse = mean_squared_error(y_test, y_test_pred) ** 0.5
        mae = mean_absolute_error(y_test, y_test_pred)
        r2 = r2_score(y_test, y_test_pred)
        results.update({
            "test_rmse": round(rmse, 4),
            "test_mae": round(mae, 4),
            "test_r2": round(r2, 4),
        })
        
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"R²: {r2:.4f}")

    return results


def monitor_cpu(interval, usage_list, stop_flag):
    psutil.cpu_percent(interval=None) 
    while not stop_flag.is_set():
        usage = psutil.cpu_percent(interval=interval) 
        usage_list.append(usage)

def run_xgboost_pipeline(dataset_name):
    (X_train, y_train), (X_val, y_val), task_type = prepare_data_np(dataset_name)
    model = build_model(task_type)

    cpu_usages = []
    stop_flag = threading.Event()
    monitor_thread = threading.Thread(target=monitor_cpu, args=(0.1, cpu_usages, stop_flag))

    start_time = time.time()
    monitor_thread.start()
    time.sleep(0.1) 

    model.fit(X_train, y_train)

    stop_flag.set()
    monitor_thread.join()
    train_time = time.time() - start_time

    avg_cpu_usage = sum(cpu_usages) / len(cpu_usages) if cpu_usages else 0

    results = evaluate_model(model, X_val, y_val, task_type, train_time, avg_cpu_usage)
    return model, results


In [ ]:
dataset_name = "adult"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 999
binary classification
(30162, 15) (30162,)
(15060, 15) (15060,)

Evaluation Results:
Accuracy: 0.8491
F1 Score: 0.7845
AUC: 0.9030

Final Metrics Summary:
train_time_seconds: 1.1723
cpu_usage: 82.8091
test_accuracy: 0.8491
test_f1: 0.7845
test_auc: 0.903


In [ ]:
dataset_name = "adult"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 888
binary classification
(30162, 15) (30162,)
(15060, 15) (15060,)

Evaluation Results:
Accuracy: 0.8509
F1 Score: 0.7871
AUC: 0.9025

Final Metrics Summary:
train_time_seconds: 1.039
cpu_usage: 80.52
test_accuracy: 0.8509
test_f1: 0.7871
test_auc: 0.9025


In [ ]:
dataset_name = "adult"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 777
binary classification
(30162, 15) (30162,)
(15060, 15) (15060,)

Evaluation Results:
Accuracy: 0.8515
F1 Score: 0.7880
AUC: 0.9020

Final Metrics Summary:
train_time_seconds: 1.0606
cpu_usage: 73.17
test_accuracy: 0.8515
test_f1: 0.788
test_auc: 0.902


In [ ]:
dataset_name = "bank+marketing"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 999
binary classification
(32950, 20) (32950,)
(8238, 20) (8238,)

Evaluation Results:
Accuracy: 0.9201
F1 Score: 0.7787
AUC: 0.9489

Final Metrics Summary:
train_time_seconds: 1.1207
cpu_usage: 83.01
test_accuracy: 0.9201
test_f1: 0.7787
test_auc: 0.9489


In [ ]:
dataset_name = "covtype"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 999
multi-class classification
(464809, 12) (464809,)
(116203, 12) (116203,)

Evaluation Results:
Accuracy: 0.9648
F1 Score: 0.9392
AUC can't be calculated: multi_class must be in ('ovo', 'ovr')

Final Metrics Summary:
train_time_seconds: 69.6606
cpu_usage: 98.6259
test_accuracy: 0.9648
test_f1: 0.9392
test_auc: nan


In [ ]:
dataset_name = "poker"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 999
multi-class classification
(25010, 10) (25010,)
(1000000, 10) (1000000,)

Evaluation Results:
Accuracy: 0.6115
F1 Score: 0.1759
AUC can't be calculated: multi_class must be in ('ovo', 'ovr')

Final Metrics Summary:
train_time_seconds: 1.2795
cpu_usage: 73.8333
test_accuracy: 0.6115
test_f1: 0.1759
test_auc: nan


In [ ]:
dataset_name = "wine"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 999
multi-class classification
(2558, 11) (2558,)
(640, 11) (640,)

Evaluation Results:
RMSE: 0.2896
MAE: 0.1843
R²: 0.8710

Final Metrics Summary:
train_time_seconds: 0.5358
cpu_usage: 67.16
test_rmse: 0.2896
test_mae: 0.1843
test_r2: 0.871


In [ ]:
dataset_name = "california"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 999
regression
(16512, 8) (16512,)
(4128, 8) (4128,)

Evaluation Results:
RMSE: 0.5117
MAE: 0.3307
R²: 0.8126

Final Metrics Summary:
train_time_seconds: 3.2111
cpu_usage: 92.9846
test_rmse: 0.5117
test_mae: 0.3307
test_r2: 0.8126


In [ ]:
dataset_name = "california"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 888
regression
(16512, 8) (16512,)
(4128, 8) (4128,)

Evaluation Results:
RMSE: 0.5149
MAE: 0.3318
R²: 0.7966

Final Metrics Summary:
train_time_seconds: 3.505
cpu_usage: 93.8852
test_rmse: 0.5149
test_mae: 0.3318
test_r2: 0.7966


In [ ]:
dataset_name = "california"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 777
regression
(16512, 8) (16512,)
(4128, 8) (4128,)

Evaluation Results:
RMSE: 0.4994
MAE: 0.3204
R²: 0.8065

Final Metrics Summary:
train_time_seconds: 3.3156
cpu_usage: 90.1462
test_rmse: 0.4994
test_mae: 0.3204
test_r2: 0.8065


In [ ]:
dataset_name = "creditcard"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 999
binary classification
(227845, 30) (227845,)
(56962, 30) (56962,)

Evaluation Results:
Accuracy: 0.9996
F1 Score: 0.9325
AUC: 0.9630

Final Metrics Summary:
train_time_seconds: 72.3207
cpu_usage: 97.544
test_accuracy: 0.9996
test_f1: 0.9325
test_auc: 0.963


In [ ]:
dataset_name = "churn"  
_, metrics = run_xgboost_pipeline(dataset_name)
print("\nFinal Metrics Summary:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Seeding: 999
binary classification
(5625, 19) (5625,)
(1407, 19) (1407,)

Evaluation Results:
Accuracy: 0.7946
F1 Score: 0.7149
AUC: 0.8179

Final Metrics Summary:
train_time_seconds: 0.4158
cpu_usage: 60.65
test_accuracy: 0.7946
test_f1: 0.7149
test_auc: 0.8179
